### 🔹 Importación de librerías  
En esta sección cargamos las librerías necesarias para el manejo de datos.  
`pandas` nos permite trabajar con archivos CSV y realizar operaciones de integración, limpieza y análisis.

In [ ]:
import pandas as pd

# Cargar el dataset combinado
data = pd.read_csv("Combined_Dataset.csv")

print("\n=== 1. Vista general del dataset ===")
print(data.info())

## 2. Vista preliminar del contenido

Revisamos las primeras 5 filas para observar la estructura básica del dataset, ejemplos de valores reales y posibles inconsistencias.  
Esto ayuda a validar que la concatenación de los datasets se realizó correctamente.

In [ ]:
print("\n=== 2. Primeras 5 filas ===")
print(data.head())

## 3. Distribución de clases (ham vs phishing)

Para evaluar el balance del dataset, contamos cuántas instancias pertenecen a cada clase (`label`):
- `0` = correo legítimo (ham)
- `1` = phishing / fraudulento

Esto es importante para determinar si se requieren técnicas de balanceo más adelante.

In [ ]:
print("\n=== 3. Distribución de clases ===")
print(data['label'].value_counts())

## 4. Análisis de valores nulos

Revisamos la cantidad total de valores faltantes por columna y su porcentaje relativo.  
Identificar estos valores es fundamental para la etapa de limpieza y feature engineering.

In [ ]:
print("\n=== 4. Valores nulos por columna ===")
print(data.isna().sum())

print("\n=== 5. Porcentaje de valores nulos ===")
print((data.isna().mean()*100).round(2))

## 5. Detección de duplicados

Buscamos duplicados tanto:
- por el cuerpo del mensaje (`body`)  
- como duplicados completos de filas

Esto permite depurar el dataset y evitar sesgos durante el modelado.

In [ ]:
print("\n=== 6. Duplicados ===")
print("Duplicados por cuerpo:", data.duplicated(subset=['body']).sum())
print("Duplicados completos:", data.duplicated().sum())

## 6. Características de longitud de texto

Creamos dos nuevas variables:
- `subject_len`: longitud del asunto  
- `body_len`: longitud del cuerpo del mensaje

Estas características son útiles para detectar patrones típicos de correos maliciosos (por ejemplo, cuerpos muy extensos o muy cortos).

In [ ]:
print("\n=== 7. Longitud de subject y body ===")
data['subject_len'] = data['subject'].astype(str).apply(len)
data['body_len'] = data['body'].astype(str).apply(len)
print(data[['subject_len', 'body_len']].describe())

## 7. Conteo aproximado de URLs por correo

Calculamos la cantidad de URLs presentes en cada correo a partir de la columna `urls`.  
Esto ayuda a identificar campañas de phishing que contienen múltiples enlaces sospechosos.

In [ ]:
print("\n=== 8. Conteo de URLs ===")
data['num_urls'] = data['urls'].astype(str).apply(lambda x: len(str(x).split()) if x != 'nan' else 0)
print(data['num_urls'].describe())

## 8. Guardado de dataset enriquecido

Guardamos el dataset actualizado con las nuevas características (`subject_len`, `body_len`, `num_urls`) en un nuevo archivo llamado `Combined_Dataset_Features.csv`.  
Este archivo será utilizado en la siguiente etapa de limpieza avanzada y feature engineering.

In [ ]:
# Guardamos con nuevas columnas
data.to_csv("Combined_Dataset_Features.csv", index=False)
print("\n>>> Archivo guardado: Combined_Dataset_Features.csv")